In [1]:
import nltk
import re
import string
import numpy as np
import pandas as pd

# Plotting tools
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [92]:
# Corpus with example sentences
corpus = ['A man is eating food.',
          'please make your system in such a way that will help business owner against fighting with illegal activities',
          'Have been working on removing a review that does not belong to our Inn. Have sent guest invite to redo the review but the link was bad. Guest and we are frustrated because she wrote us apologizing and the hotel she did the review for wasnt even in our town.'
          'it should be possible to delete a reviwew that is obviously unjust and wrong',
          'i do not like the fact that the pictures are changing by rotation i have now for all the rooms if i search the property the image with the bathrooms it is not good',
          'staff need to be trained so that when a guest house asks a question that they can answer',
          'expedia much much better you always take a part of the customer who may be drunk has mental problems does not meter for you',
         'we received an anonymous review which was really bad' 
          ,'please escalate this to someone who can help'
          ,'when a known issue like vcc problemsa message on the extranet would have saved me holding online for 10 min'
          ,'was tranfered then cut off now waiting to try again first guy was very helpful'
          ,'i would like to cancel genius promotion but i need to wait for long time to deal with until now'
         ]

In [93]:
# Query sentences:
queries = ['i am trying to explain the situation with the guest the guest has deliberately tried to give bad reviews this should not be allowed if the guest was unsatisfied with his stay he could have find a replacement reaching out to bookingcom but rather he continued staying here i want his reviews to be taken out checking my previous history of reviews i cannot allow'
          ,'i was told to message the request directly the reply i received then suggested i tried doing exactly what i had already done again this did not work and i am still waiting for actual assistance' 
          ]

In [95]:
from sentence_transformers import SentenceTransformer, util
import torch
#import Top2Vec
# first time, the following line has to be run in python terminal
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [96]:
top_k = 5
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use torch.topk to find the highest 5 scores
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: %.4f)" % (score))





Query: i am trying to explain the situation with the guest the guest has deliberately tried to give bad reviews this should not be allowed if the guest was unsatisfied with his stay he could have find a replacement reaching out to bookingcom but rather he continued staying here i want his reviews to be taken out checking my previous history of reviews i cannot allow

Top 5 most similar sentences in corpus:
Have been working on removing a review that does not belong to our Inn. Have sent guest invite to redo the review but the link was bad. Guest and we are frustrated because she wrote us apologizing and the hotel she did the review for wasnt even in our town.it should be possible to delete a reviwew that is obviously unjust and wrong (Score: 0.6552)
i would like to cancel genius promotion but i need to wait for long time to deal with until now (Score: 0.5553)
i do not like the fact that the pictures are changing by rotation i have now for all the rooms if i search the property the 

In [97]:
import tensorflow_hub as hub
module = "/home/yueyang/encoder/universal-sentence-encoder"
embedder = hub.load(module)
corpus_embeddings = embedder(corpus)

INFO:absl:resolver HttpCompressedFileResolver does not support the provided handle.
INFO:absl:resolver GcsCompressedFileResolver does not support the provided handle.


In [98]:
import scipy
import math

def hub_similarity(query, text, embedder):
    query_embedding = tf.nn.l2_normalize(embedder(tf.constant([query])), axis=1)
    text_embedding = tf.nn.l2_normalize(embedder(tf.constant([text])), axis=1)
    cosine_similarities = tf.reduce_sum(tf.multiply(query_embedding, text_embedding ), axis=1)
    clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
    scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
    
    return scores

In [99]:
import tensorflow as tf
from operator import itemgetter
top_k = 5

for query in queries:
    scores = []
    for text in corpus:
        score = hub_similarity(query, text, embedder)
        scores.append(score.numpy()[0])
    results = zip(corpus, scores)
    top_results = sorted(results, key= itemgetter(1), reverse = True)[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for text, score in top_results:
        print(text, "(Score: %.4f)" % (score))





Query: i am trying to explain the situation with the guest the guest has deliberately tried to give bad reviews this should not be allowed if the guest was unsatisfied with his stay he could have find a replacement reaching out to bookingcom but rather he continued staying here i want his reviews to be taken out checking my previous history of reviews i cannot allow

Top 5 most similar sentences in corpus:
Have been working on removing a review that does not belong to our Inn. Have sent guest invite to redo the review but the link was bad. Guest and we are frustrated because she wrote us apologizing and the hotel she did the review for wasnt even in our town.it should be possible to delete a reviwew that is obviously unjust and wrong (Score: 0.7384)
staff need to be trained so that when a guest house asks a question that they can answer (Score: 0.5940)
we received an anonymous review which was really bad (Score: 0.5847)
expedia much much better you always take a part of the custome